In [1]:
import gumpy as gp
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import sklearn
import os


import warnings

warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd
from sklearn.decomposition import PCA
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.lda import LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from mlxtend.classifier import StackingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



# load data step

In [4]:
def load_data(filename):
    
    #extra_overlap = 1500
    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    #此通道没有采集 置为0
    #全通道均使用时 不需要
    #for i in range(len(SAMPLE_CHANNEL)):
    #    if SAMPLE_CHANNEL[i] == 'none':
    #        data[: , i] = 0.0

    #删除前x秒和后x秒数据
    
    
    #是否进行裁剪 【如果进行裁剪 由于sen的第一次数据 将extra_overlap调整为1500】
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    
    #data = np.concatenate((data , data[ -extra_overlap : , :]) , axis=0)
    #
    #data_filter = butter_worth(data , 0.5 , 40 , order=3)
    #
    #return data_filter[extra_overlap : , :] #将边界效应去掉

    return data

In [5]:
def separate(data , label , overlap_length = 128):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [6]:
def train_val(data , ratio = 0.9):
    '''
    将数据分为 训练集 和 验证集
    '''
    
    seg = int(ratio * data.shape[0])
    
    return data[ : seg] , data[seg : ]

def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames

def combine(freq = 10):
    '''
    训练数据与验证数据
    :freq: 指定闪烁的频率
    
    '''
    
    if freq not in [10 , 15 , 20 , 25]:
        print('freq must in 10,15,20,25')
        return 
    
    ratio = 0.9 #训练集的占比
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/8/%s/' % freq) )
    person_9_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/9/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('real_data/eeg_final/circle/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('real_data/eeg_final/circle/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('real_data/eeg_final/circle/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('real_data/eeg_final/circle/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('real_data/eeg_final/circle/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('real_data/eeg_final/circle/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('real_data/eeg_final/circle/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('real_data/eeg_final/circle/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('real_data/eeg_final/circle/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    person_9 = np.concatenate([load_data('real_data/eeg_final/circle/9/%s/' % freq + filename) for filename in person_9_filenames] , axis = 0)
    
    person_0_train , person_0_val = train_val(person_0)
    person_1_train , person_1_val = train_val(person_1)
    person_2_train , person_2_val = train_val(person_2)
    person_3_train , person_3_val = train_val(person_3)
    person_4_train , person_4_val = train_val(person_4)
    person_5_train , person_5_val = train_val(person_5)
    person_6_train , person_6_val = train_val(person_6)
    person_7_train , person_7_val = train_val(person_7)
    person_8_train , person_8_val = train_val(person_8)
    person_9_train , person_9_val = train_val(person_9)
    
    #数据分段阶段
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0_train , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1_train , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2_train , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3_train , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4_train , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5_train , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6_train , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7_train , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8_train , label = 8 , overlap_length=overlap_length)
    train_person_data_9 , train_person_labels_9 = separate(person_9_train , label = 9 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 train_person_data_9 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                   train_person_labels_9 ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
    
    #============
    #验证数据分段
    val_person_data_0 , val_person_labels_0 = separate(person_0_val , label = 0 , overlap_length=0)
    val_person_data_1 , val_person_labels_1 = separate(person_1_val , label = 1 , overlap_length=0)
    val_person_data_2 , val_person_labels_2 = separate(person_2_val , label = 2 , overlap_length=0)
    val_person_data_3 , val_person_labels_3 = separate(person_3_val , label = 3 , overlap_length=0)
    val_person_data_4 , val_person_labels_4 = separate(person_4_val , label = 4 , overlap_length=0)
    val_person_data_5 , val_person_labels_5 = separate(person_5_val , label = 5 , overlap_length=0)
    val_person_data_6 , val_person_labels_6 = separate(person_6_val , label = 6 , overlap_length=0)
    val_person_data_7 , val_person_labels_7 = separate(person_7_val , label = 7 , overlap_length=0)
    val_person_data_8 , val_person_labels_8 = separate(person_8_val , label = 8 , overlap_length=0)
    val_person_data_9 , val_person_labels_9 = separate(person_9_val , label = 9 , overlap_length=0)
    
    #合并数据
    val_data = np.concatenate((val_person_data_0 , val_person_data_1 , val_person_data_2 ,
                               val_person_data_3 , val_person_data_4 , val_person_data_5 ,
                               val_person_data_6 , val_person_data_7 , val_person_data_8 ,
                               val_person_data_9 ))
    
    val_labels = np.concatenate((val_person_labels_0 , val_person_labels_1 , val_person_labels_2 ,
                                 val_person_labels_3 , val_person_labels_4 , val_person_labels_5 ,
                                 val_person_labels_6 , val_person_labels_7 , val_person_labels_8 ,
                                 val_person_labels_9 ))
    
    #产生索引并置乱
    idx_val_data = list(range(val_data.shape[0]))
    np.random.shuffle(idx_val_data)

    #将训练数据置乱
    val_data = val_data[idx_val_data]
    val_labels = val_labels[idx_val_data]
    
    return train_data , train_labels , val_data , val_labels


In [7]:
def shuffle(train_data , train_labels , val_data , val_labels):
    #置乱一次数据
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)
    
    idx_val_data = list(range(val_data.shape[0]))
    np.random.shuffle(idx_val_data)
    
    return train_data[idx_train_data] , train_labels[idx_train_data] , val_data[idx_val_data] , val_labels[idx_val_data]

In [8]:
#train_X_ , train_y , val_X_ , val_y = combine(freq = 10) #10 15 20 25 hz

In [9]:
#如果没有进行前后裁剪 则输出数据会变多

#print(train_X_.shape , train_y.shape , val_X_.shape , val_y.shape)

In [10]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

In [11]:
def alpha_subBP_features(data):
    alpha1 = butter_worth(data , 8.5 , 11.5)
    alpha2 = butter_worth(data , 9.0 , 12.5)    
    alpha3 = butter_worth(data , 9.5 , 13.5)   #11.5 后
    alpha4 = butter_worth(data , 8.0 , 10.5)   
    
    return np.array([alpha1 , alpha2 , alpha3 , alpha4])

def beta_subBP_features(data):
    beta1 = butter_worth(data , 15.0 , 30.0) #14.0 前
    beta2 = butter_worth(data , 16.0 , 17.0)    
    beta3 = butter_worth(data , 17.0 , 18.0)    
    beta4 = butter_worth(data , 18.0 , 19.0)    
    
    return np.array([beta1 , beta2 , beta3 , beta4])

def powermean(data):
    #官方demo跳4秒 前4秒为准备阶段
    return np.power(data[ : , 0] , 2).mean(), \
            np.power(data[ : , 1] , 2).mean(), \
            np.power(data[ : , 2] , 2).mean(), \
            np.power(data[ : , 3] , 2).mean(), \
            np.power(data[ : , 4] , 2).mean(), \
            np.power(data[ : , 5] , 2).mean(), \
            np.power(data[ : , 6] , 2).mean(), \
            np.power(data[ : , 7] , 2).mean(), \
            np.power(data[ : , 8] , 2).mean(), \
            np.power(data[ : , 9] , 2).mean(), \
            np.power(data[ : , 10] , 2).mean(), \
            np.power(data[ : , 11] , 2).mean(), \
            np.power(data[ : , 12] , 2).mean(), \
            np.power(data[ : , 13] , 2).mean(), \
            np.power(data[ : , 14] , 2).mean(), \
            np.power(data[ : , 15] , 2).mean()       

In [12]:
def log_subBP_feature_extraction(alpha , beta):
    #alpha
    power_1_a = powermean(alpha[0])
    power_2_a = powermean(alpha[1])
    power_3_a = powermean(alpha[2])
    power_4_a = powermean(alpha[3])
    
    #beta
    power_1_b = powermean(beta[0])
    power_2_b = powermean(beta[1])
    power_3_b = powermean(beta[2])
    power_4_b = powermean(beta[3])
    
    X= np.array(
        [np.log(power_1_a) ,
         np.log(power_2_a) ,
         np.log(power_3_a) ,
         np.log(power_4_a) ,
         np.log(power_1_b) ,
         np.log(power_2_b) ,
         np.log(power_3_b) ,
         np.log(power_4_b)
        ]
        ).flatten()

    return X

In [13]:
def feature_extraction_sub_band_power(data):
    n_features = 128
    X = np.zeros((data.shape[0] , n_features))
    
    for i , datum in enumerate(data):
        alpha = alpha_subBP_features(datum)
        beta = beta_subBP_features(datum)
            
        X[i, :] = log_subBP_feature_extraction(alpha , beta)

    return X

In [25]:
def con_mat(_feature , _labels , model):
    '''
    打印训练结果
    '''
    
    print('val score:%f' % model.score(_feature , _labels))
    print('real')
    
    print(confusion_matrix( _labels , model.predict(_feature) ))
    print(classification_report( _labels , model.predict(_feature) ))

def con_mat_heatmap(_feature , _labels , model , color , png_path):
    
    _labels_hat = model.predict(_feature)
    
    mat = confusion_matrix( _labels , _labels_hat )
    
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False , cmap=color )#,
            #xticklabels=faces.target_names,
            #yticklabels=faces.target_names)
    
    acc = accuracy_score(_labels , _labels_hat)
    recall = recall_score(_labels , _labels_hat , average='macro')
    f1 = f1_score(_labels , _labels_hat , average='macro')
    
    plt.xlabel('accuracy:%.2f recall:%.2f f1:%.2f' % (acc , recall , f1) )
    # plt.ylabel('predicted label');
    plt.savefig(png_path) #保存起来
    plt.close()

def feature_selection(data , labels , model , num_features , cv=10):
    '''
    :model: classify model
    :num_features: features count you expect(integer or tuple)
    '''
    
    '''[8 20]'''
    
    sfs = SequentialFeatureSelector(model , k_features=num_features , cv=cv , verbose = 2 , n_jobs=-1) #all cpu cores
    
    sfs.fit(data , labels)
    
    #最优秀的特征索引
    return sfs.k_feature_idx_


def choose_common_feature_idx(data , labels , classifiers , use_ratio = 0.25 , num_features = 10 , num_features_threshold = 8):
    '''
    sub_band_power使用该函数 进行筛选特征
    :use_ratio: 0.25 ~= 360/1500
    :num_features:integer or tuple 期望的特征数量（待选择的数量）
    :min_num_features: 特征数量阈值 小于时 停止选择
    选择适合所有分类器的特征索引值
    集合 与 运算
    '''
    
    #将特征选择需要使用的数据 0.25 进行置乱
    idxes_ratio = np.random.randint(0 , data.shape[0] , size = int(use_ratio * data.shape[0]) )
    
    
    data_shuffle = data #[idxes_ratio]
    labels_shuffle = labels #[idxes_ratio]
    
    feature_idxes = set(list(range(data.shape[1]))) #初始化为所有的特征索引值
    
    #============
    random_idxes = np.random.randint(0 , data.shape[1] , size = 25)
    
    data_shuffle = data_shuffle[: , random_idxes]
    labels_shuffle = labels_shuffle #保持完整性
    #============
    
    for classifier in classifiers:
        
        idx = feature_selection(data_shuffle , labels_shuffle , classifier , num_features)
        idx = set(idx)
        
        #寻找共同的特征索引
        #寻找之前先测试 如果小于阈值 直接停止
        if len(feature_idxes & idx) < num_features_threshold:
            break
            
        feature_idxes = feature_idxes & idx
        
        break
        
    return np.array(list(feature_idxes))

#新增函数 计算f1分数
def f1_score_(_feature , _labels , model):
    
    _labels_hat = model.predict(_feature)
    
    accu = accuracy_score(_labels , _labels_hat )
    precision = precision_score(_labels , _labels_hat , average='macro')
    recall = recall_score(_labels , _labels_hat , average='macro')
    f1 = f1_score(_labels , _labels_hat , average='macro')
    kappa = cohen_kappa_score(_labels , _labels_hat)
    
    return accu , precision, recall , f1 , kappa

def roc_auc(real_label , score):
    return roc_auc_score( label_binarize(real_label , classes=[0,1,2,3,4,5,6,7,8,9]) , score , average='micro')

def itrr(acc , N=10):
    return np.log2(N) + acc*np.log2(acc) + (1-acc)*np.log2((1-acc)/(N-1))

In [29]:
#初始化所有分类器
svc = SVC(probability=True)

lgbm = lightgbm.LGBMClassifier()
#xgb = xgboost.XGBClassifier()
#gbc = GradientBoostingClassifier()
#rf =  RandomForestClassifier()
#adaboost = AdaBoostClassifier()
#knn = KNeighborsClassifier()
#dt = DecisionTreeClassifier()
#lda = LDA()
#nb = GaussianNB()
#mlp = MLPClassifier()

classifiers = [svc]

In [27]:
#classifier = svc
#
#train_X_ , train_y , val_X_ , val_y = combine(freq=10)
#
##提取特征
#train_X = feature_extraction_sub_band_power(train_X_)
#val_X = feature_extraction_sub_band_power(val_X_)
#
#classifier.fit(train_X , train_y)
#
#con_mat_heatmap(val_X , val_y , classifier , color=None , png_path='./1.png')
#
#acc , _ , rec , f1s , _ = f1_score_(val_X , val_y , classifier)
#
#auc = roc_auc(val_y , classifier.predict_proba(val_X))
#
#itr = itrr(acc) #BCI系统的速率
#
#print('[%.6f %.6f %.6f %.6f %.6f]' % (acc ,  rec , f1s , auc , itr ) )

[0.580000 0.580000 0.519877 0.954933 1.009106]


In [31]:
for freq in [10 , 15 , 20 , 25]:
    
    for (i , classifier) in enumerate(classifiers):
        acc_s = []
        rec_s = []
        f1s_s = []
        auc_s = []
        itr_s = []
                

        for t in range(1): #循环20次 取平均值 作为最终结果

            train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

            #提取特征
            train_X = feature_extraction_sub_band_power(train_X_)
            val_X = feature_extraction_sub_band_power(val_X_)
            
            classifier.fit(train_X , train_y)

            acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , classifier)
            auc = roc_auc(val_y , classifier.predict_proba(val_X))
            itr = itrr(acc)
            
            con_mat_heatmap(val_X , val_y , classifier , color=None , png_path='./%d_%d.png' % (freq , i) )
            
            acc_s.append(acc)
            rec_s.append(rec)
            f1s_s.append(f1s)
            auc_s.append(auc)
            itr_s.append(itr)
            
        print('[%d %d] %.6f %.6f %.6f %.6f %.6f' % (freq , i ,  np.mean(acc_s), np.mean(rec_s) , np.mean(f1s_s) , np.mean(itr_s) ,  np.mean(itr_s) ) )

[10 0] 0.700000 0.700000 0.685880 1.489660 1.489660
[15 0] 0.860000 0.860000 0.858131 2.293900 2.293900
[20 0] 0.860000 0.860000 0.856465 2.293900 2.293900
[25 0] 0.800000 0.800000 0.776061 1.966015 1.966015
